# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "../output_data/cities.csv"
data_file_df =pd.read_csv(data_file)
data_file_df.head()

,Unnamed: 0,Country,City,Lat,Lng,Temp F,Date,Humidity,Max Temp,Wind Speed,Cloudiness
0,0,RU,Severo-Kuril'sk,50.68,156.12,34.39,1587240637,88,34.39,10.07,9
1,1,RU,Dikson,73.51,80.55,35.24,1587240493,89,35.24,19.10,100
2,2,IT,Laas,46.62,10.70,55.40,1587240649,58,55.40,2.24,40
3,3,TO,Vaini,-21.20,-175.20,77.00,1587240488,83,77.00,10.29,40
4,4,NZ,Bluff,-46.60,168.33,43.00,1587240576,91,43.00,5.01,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = data_file_df[["Lat","Lng"]]
humidity = data_file_df["Humidity"].astype(float)
fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df=data_file_df.loc[(data_file_df['Max Temp']>70) & (data_file_df['Max Temp']<85)\
                          &(data_file_df['Cloudiness']<40)&(data_file_df['Humidity']<40)&(data_file_df['Wind Speed']<10)]                
hotel_df.count()

Unnamed: 0    6
Country       6
City          6
Lat           6
Lng           6
Temp F        6
Date          6
Humidity      6
Max Temp      6
Wind Speed    6
Cloudiness    6
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name']=""
hotel_df

C:\Users\wsw22\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Country,City,Lat,Lng,Temp F,Date,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel Name
330,330,AR,Rivadavia,-33.19,-68.46,82.40,1587240736,32,82.40,6.93,0,
360,360,IN,Bilkha,21.43,70.60,81.37,1587241018,38,81.37,4.74,9,
449,449,TD,Biltine,14.53,20.92,80.89,1587241031,13,80.89,9.08,0,
472,472,EG,Aswān,24.09,32.91,71.06,1587240692,23,71.06,8.25,6,
521,521,SD,Kutum,14.20,24.67,70.56,1587240627,14,70.56,9.55,7,
541,541,NE,Bilma,18.69,12.92,83.82,1587241043,12,83.82,5.03,0,


In [6]:
params = {
    "radius":5000,
    "types": "lodging",
    "key":g_key,
    }

hotel_name = []

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    hotel_name.append(response['results'][0]['name'])
    
    hotel = response['results'][0]['name']
#     rating = response['results'][0]['rating']
    
    print(f'Found {hotel}.  Getting Data.')

hotel_name

Found Posada Del Fin Del Mundo.  Getting Data.
Found Shri Sthanakvasi Jain Sangh nivJagya.  Getting Data.
Found Hôtel Trois Étoiles de Biltine.  Getting Data.
Found Nuba Nile.  Getting Data.
Found Dibbo areaمنطقة دبو.  Getting Data.
Found Expeditions Ténére voyages.  Getting Data.


['Posada Del Fin Del Mundo',
 'Shri Sthanakvasi Jain Sangh nivJagya',
 'Hôtel Trois Étoiles de Biltine',
 'Nuba Nile',
 'Dibbo areaمنطقة دبو',
 'Expeditions Ténére voyages']

In [7]:
pprint(response)

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 18.6929789, 'lng': 12.9176518},
                           'viewport': {'northeast': {'lat': 18.6943387802915,
                                                      'lng': 12.9190450802915},
                                        'southwest': {'lat': 18.6916408197085,
                                                      'lng': 12.9163471197085}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': 'b94d8494958d2125c0335834e552772132ae5d48',
              'name': 'Expeditions Ténére voyages',
              'place_id': 'ChIJK367YL5dcBERK29v6zybUaw',
              'plus_code': {'compound_code': 'MWV9+53 Bilma, Niger',
                            'global_code': '7FCJMWV9+53'},
              'rating': 4.7,
              'reference': 'ChIJK367YL5dcBERK29v6zybUaw',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establi

In [8]:
hotel_df["Hotel Name"] = hotel_name
hotel_df


C:\Users\wsw22\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Country,City,Lat,Lng,Temp F,Date,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel Name
330,330,AR,Rivadavia,-33.19,-68.46,82.40,1587240736,32,82.40,6.93,0,Posada Del Fin Del Mundo
360,360,IN,Bilkha,21.43,70.60,81.37,1587241018,38,81.37,4.74,9,Shri Sthanakvasi Jain Sangh nivJagya
449,449,TD,Biltine,14.53,20.92,80.89,1587241031,13,80.89,9.08,0,Hôtel Trois Étoiles de Biltine
472,472,EG,Aswān,24.09,32.91,71.06,1587240692,23,71.06,8.25,6,Nuba Nile
521,521,SD,Kutum,14.20,24.67,70.56,1587240627,14,70.56,9.55,7,Dibbo areaمنطقة دبو
541,541,NE,Bilma,18.69,12.92,83.82,1587241043,12,83.82,5.03,0,Expeditions Ténére voyages


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer=gmaps.symbol_layer(
    locations, fill_color='red', scale=3,
    info_box_content=hotel_info)


# Display Map
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))